# Amazon SageMaker Ground Truth for high quality labeling



# Introduction

This sample notebook takes you through creation of a Ground Truth labeling job.


### Steps
To create a Ground Truth job we will perform the following steps:
* Create S3 bucket which will hold the images as well as job related data
* Set proper CORS permissions on the bucket
* Upload the images to be labled
* Define labeling job template
* Create labeling job using private workforce

In [ ]:
%matplotlib inline
import os
import base64
import json
import time
import imageio
import boto3
import sagemaker
import time
from os import listdir
from os.path import isfile, join

### <font color='red'> USER INPUT REQUIRED </font> 

In [ ]:
BUCKET = <<INPUT UNIQUE BUCKET NAME IN QUOTES>> # Change this to your bucket
private_workteam_arn = <<INPUT PRIVATE TEAM ARN IN QUOTES>># Change this to your private team arn
EXP_NAME = "aim406"  # Any valid S3 prefix.

In [ ]:
# Make sure the bucket is in the same region as this notebook.
role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name
s3 = boto3.client("s3")
bucket_region = s3.head_bucket(Bucket=BUCKET)["ResponseMetadata"]["HTTPHeaders"][
    "x-amz-bucket-region"
]
assert (
    bucket_region == region
), "Your S3 bucket {} and this notebook need to be in the same region.".format(BUCKET)

In [ ]:
# Define the configuration rules
cors_configuration = {
    'CORSRules': [{
        'AllowedHeaders': [],
        'AllowedMethods': ['GET'],
        'AllowedOrigins': ['*'],
        'ExposeHeaders': []
    }]
}


# Set the CORS configuration
s3.put_bucket_cors(Bucket=BUCKET,
                   CORSConfiguration=cors_configuration)

# Upload pics to S3 and create manifest

In [ ]:
mypath="Logos"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Create and upload the input manifest.
manifest_name = "input.manifest"
with open(manifest_name, "w") as f:
    for img_id in onlyfiles:
        img_path = "s3://{}/{}/images/{}".format(BUCKET, EXP_NAME, img_id)
        f.write('{"source-ref": "' + img_path + '"}\n')
        s3.upload_file(mypath+'/'+img_id, BUCKET, EXP_NAME + "/images/" + img_id)
        
s3.upload_file(manifest_name, BUCKET, EXP_NAME + "/" + manifest_name)

After running the cell above, you should be able to go to `s3://BUCKET/EXP_NAME/images` in the [S3 console](https://console.aws.amazon.com/s3/) and see the images which need to be labled

## Specify the categories

To run an object detection labeling job, you must decide on a set of classes the annotators can choose from. At the moment, Ground Truth only supports annotating one OD class at a time. In our case, the singleton class list is simply `["Bird"]`.  To work with Ground Truth, this list needs to be converted to a .json file and uploaded to the S3 `BUCKET`.

In [ ]:
CLASS_LIST = ['DHL','McDonalds', 'RedBull']
print("Label space is {}".format(CLASS_LIST))

json_body = {"labels": [{"label": label} for label in CLASS_LIST]}
with open("class_labels.json", "w") as f:
    json.dump(json_body, f)

s3.upload_file("class_labels.json", BUCKET, EXP_NAME + "/class_labels.json")

You should now see `class_labels.json` in `s3://BUCKET/EXP_NAME/`.

## Create the instruction template

Part or all of your images will be annotated by human annotators. It is **essential** to provide good instructions. Good instructions are:
1. Concise. We recommend limiting verbal/textual instruction to two sentences and focusing on clear visuals.
2. Visual. In the case of object detection, we recommend providing several labeled examples with different numbers of boxes.

When used through the AWS Console, Ground Truth helps you create the instructions using a visual wizard. When using the API, you need to create an HTML template for your instructions. Below, we prepare a very simple but effective template and upload it to your S3 bucket.

NOTE: If you use any images in your template (as we do), they need to be publicly accessible. You can enable public access to files in your S3 bucket through the S3 Console, as described in [S3 Documentation](https://docs.aws.amazon.com/AmazonS3/latest/user-guide/set-object-permissions.html). 

#### Testing your instructions
**It is very easy to create broken instructions.** This might cause your labeling job to fail. However, it might also cause your job to complete with meaningless results if, for example, the annotators have no idea what to do or the instructions are misleading. At the moment the only way to test the instructions is to run your job in a private workforce. This is a way to run a mock labeling job for free. We describe how in [Verify your task using a private team [OPTIONAL]](#Verify-your-task-using-a-private-team-[OPTIONAL]).

It is helpful to show examples of correctly labeled images in the instructions. The following code block produces several such examples for our dataset and saves them in `s3://BUCKET/EXP_NAME/`.

In [ ]:
with open('sample.jpg', 'rb') as instructions:
    instructions_uri = base64.b64encode(instructions.read()).decode('utf-8').replace('\n', '')

In [ ]:
from IPython.core.display import HTML, display


def make_template(test_template=False, save_fname="instructions.template"):
    template = r"""<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
    <crowd-form>
      <crowd-bounding-box
        name="boundingBox"
        src="{{{{ task.input.taskObject | grant_read_access }}}}"
        header="Dear Annotator, please draw a tight box around each company logo you see (if there are more than one, draw boxes around each). Thank you!"
        labels="{labels_str}"
      >
        <full-instructions header="Please annotate each logo.">

    <ol>
        <li><strong>Inspect</strong> the image</li>
        <li><strong>Determine</strong> if the specified label is/are visible in the picture.</li>
        <li><strong>Outline</strong> each instance of the specified label in the image using the provided “Box” tool.</li>
    </ol>
    <ul>
        <li>Boxes should fit tight around each object</li>
        <li>Do not include parts of the object are overlapping or that cannot be seen, even though you think you can interpolate the whole shape.</li>
        <li>Avoid including shadows.</li>
        <li>If the target is off screen, draw the box up to the edge of the image.</li>
    </ul>

        </full-instructions>
        <short-instructions>
        <img src="data:image/png;base64,{instructions_uri}" style="max-width:100%">
        </short-instructions>
      </crowd-bounding-box>
    </crowd-form>
    """.format(
        class_name='Company Logo',
        instructions_uri=instructions_uri,
        labels_str=str(CLASS_LIST)
        if test_template
        else "{{ task.input.labels | to_json | escape }}",
    )
    with open(save_fname, "w") as f:
        f.write(template)


make_template(test_template=True, save_fname="instructions.html")
make_template(test_template=False, save_fname="instructions.template")
s3.upload_file("instructions.template", BUCKET, EXP_NAME + "/instructions.template")

You should now be able to find your template in `s3://BUCKET/EXP_NAME/instructions.template`.

In [ ]:
# Specify ARNs for resources needed to run an object detection job.
ac_arn_map = {
    "us-west-2": "081040173940",
    "us-east-1": "432418664414",
    "us-east-2": "266458841044",
    "eu-west-1": "568282634449",
    "ap-northeast-1": "477331159723",
}

prehuman_arn = "arn:aws:lambda:{}:{}:function:PRE-BoundingBox".format(region, ac_arn_map[region])
acs_arn = "arn:aws:lambda:{}:{}:function:ACS-BoundingBox".format(region, ac_arn_map[region])
labeling_algorithm_specification_arn = "arn:aws:sagemaker:{}:027400017018:labeling-job-algorithm-specification/object-detection".format(
    region
)
workteam_arn = "arn:aws:sagemaker:{}:394669845002:workteam/public-crowd/default".format(region)

## Submit the Ground Truth job request
The API starts a Ground Truth job by submitting a request.

In [ ]:
VERIFY_USING_PRIVATE_WORKFORCE = True
USE_AUTO_LABELING = True
CLASS_NAME='Company Logo'

task_description = "Dear Annotator, please draw a box around each {}. Thank you!".format(CLASS_NAME)
task_keywords = ["image", "object", "detection"]
task_title = "Please draw a box around each {}.".format(CLASS_NAME)
job_name = "reinv-gt-" + str(int(time.time()))

human_task_config = {
    "AnnotationConsolidationConfig": {
        "AnnotationConsolidationLambdaArn": acs_arn,
    },
    "PreHumanTaskLambdaArn": prehuman_arn,
    "MaxConcurrentTaskCount": 10,  # Number of images that will be sent at a time to the workteam.
    "NumberOfHumanWorkersPerDataObject": 1,  # We will obtain and consolidate at this many human annotations for each image.
    "TaskAvailabilityLifetimeInSeconds": 21600,  # Your workteam has 6 hours to complete all pending tasks.
    "TaskDescription": task_description,
    "TaskKeywords": task_keywords,
    "TaskTimeLimitInSeconds": 300,  # Each image must be labeled within 5 minutes.
    "TaskTitle": task_title,
    "UiConfig": {
        "UiTemplateS3Uri": "s3://{}/{}/instructions.template".format(BUCKET, EXP_NAME),
    },
}


human_task_config["WorkteamArn"] = private_workteam_arn

ground_truth_request = {
    "InputConfig": {
        "DataSource": {
            "S3DataSource": {
                "ManifestS3Uri": "s3://{}/{}/{}".format(BUCKET, EXP_NAME, manifest_name),
            }
        },
        "DataAttributes": {
            "ContentClassifiers": ["FreeOfPersonallyIdentifiableInformation", "FreeOfAdultContent"]
        },
    },
    "OutputConfig": {
        "S3OutputPath": "s3://{}/{}/output/".format(BUCKET, EXP_NAME),
    },
    "HumanTaskConfig": human_task_config,
    "LabelingJobName": job_name,
    "RoleArn": role,
    "LabelAttributeName": "category",
    "LabelCategoryConfigS3Uri": "s3://{}/{}/class_labels.json".format(BUCKET, EXP_NAME),
}

if USE_AUTO_LABELING:
    ground_truth_request["LabelingJobAlgorithmsConfig"] = {
        "LabelingJobAlgorithmSpecificationArn": labeling_algorithm_specification_arn
    }

sagemaker_client = boto3.client("sagemaker")
sagemaker_client.create_labeling_job(**ground_truth_request)